In [3]:
from secrets_pwd import api_key, api_secret
from binance_handler import BinanceHandler

symbol = "BTCEUR"
binance_handler = BinanceHandler(api_key=api_key,api_secret=api_secret, symbol=symbol)
historical_data = binance_handler.get_historical_data("10 day ago UTC", "now")

In [4]:
import pandas as pd

def from_historical_to_df(historical_data) -> pd.DataFrame:
    df = pd.DataFrame(historical_data, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
    df['Close'] = pd.to_numeric(df['Close'], downcast="float")/10000
    df['Open'] = pd.to_numeric(df['Open'], downcast="float")/10000
    df['Volume'] = pd.to_numeric(df['Volume'], downcast="float")
    df['High'] = pd.to_numeric(df['High'], downcast="float")/10000
    df['Low'] = pd.to_numeric(df['Low'], downcast="float")/10000
    df['Quote asset volume'] = pd.to_numeric(df['Quote asset volume'], downcast="float")
    df['Taker buy base asset volume'] = pd.to_numeric(df['Taker buy base asset volume'], downcast="float")
    df['Taker buy quote asset volume'] = pd.to_numeric(df['Taker buy quote asset volume'], downcast="float")
    df['Ignore'] = df['Ignore'] == 0
    return df
    
df = from_historical_to_df(historical_data)
df

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1705492680000,3.923494,3.924778,3.923494,3.924778,0.01256,1705492739999,492.895230,9,0.00820,321.830891,False
1,1705492740000,3.924791,3.924890,3.924791,3.924890,0.01758,1705492799999,689.991783,4,0.01758,689.991783,False
2,1705492800000,3.924548,3.925959,3.923075,3.923075,0.03262,1705492859999,1280.155254,9,0.00534,209.501180,False
3,1705492860000,3.923446,3.923478,3.922294,3.922915,0.22211,1705492919999,8713.361867,21,0.08286,3250.697701,False
4,1705492920000,3.922915,3.922915,3.922335,3.922474,0.26456,1705492979999,10377.767982,16,0.02602,1020.618457,False
...,...,...,...,...,...,...,...,...,...,...,...,...
14395,1706356380000,3.855153,3.855529,3.855153,3.855483,0.06836,1706356439999,2635.624396,5,0.04243,1635.897654,False
14396,1706356440000,3.855193,3.855193,3.855193,3.855193,0.00026,1706356499999,10.023502,1,0.00000,0.000000,False
14397,1706356500000,3.855708,3.855708,3.854262,3.854262,0.01212,1706356559999,467.212786,6,0.00736,283.702601,False
14398,1706356560000,3.854262,3.854833,3.853651,3.853651,0.05614,1706356619999,2163.678171,14,0.04293,1654.528523,False


In [ ]:
import gym
import numpy as np
from gym import spaces
from typing import List, Dict

class BTCEnv(gym.Env):

    def __init__(self, binance_handler: BinanceHandler, n_previous_observations = 0, include_action = False):
        obs_dict = {
            'Open': spaces.Box(0, np.inf), 
            'High': spaces.Box(0, np.inf),
            'Low': spaces.Box(0, np.inf), 
            'Close': spaces.Box(0, np.inf), 
            'Volume': spaces.Box(0, np.inf),
            'Quote asset volume': spaces.Box(0, np.inf),
            'Taker buy base asset volume': spaces.Box(0, np.inf),
            'Taker buy quote asset volume': spaces.Box(0, np.inf),
        }
        if include_action: obs_dict['Action'] = spaces.Discrete(3)
        self.observation_space = spaces.Dict(obs_dict)
        self.action_space = spaces.Discrete(3)

        self.binance_handler = binance_handler

    def step(self, action, sample_time):

        